In [229]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.metrics import mean_squared_error
from pathlib import Path

In [230]:
# 10GB, new reference genom

# number of sockets, threds, logical cores, cores, ram, rss, strength of cores, cpu type
# extension of dataset (reruns, use interpolations)
# 2 layer, 3 layer, 4layer
# 1, 2, 3, 4, 5 Nodes


def give_me_x_and_y_per_node(df_general_results, name):
    #take data from the df
    df = df_general_results[df_general_results['Node Name'] == name]

    return df['Data Size'].values, df['Total-time'].values

def give_me_all_x_y(df_general_results):
    return df_general_results['Data Size'].values, df_general_results['Total-time'].values

def build_big(name_list):
    result = pd.DataFrame({'Node Name': np.empty(1), 'Total-time': np.empty(1), 'Data Size': np.empty(1)})
    for path in name_list:
        test = pd.read_csv(path, sep='\t')
        test_general_results = test[['Node Name', 'Total-time', 'Data Size']]
        result = pd.concat([result, test_general_results])
    result = result.dropna(how='any')
    return result

def train_and_test(x,y):
    
    if len(x) == len(y):
        n = len(x) // 5
    else:
        print('dataset is corrupted')
    
    np.random.shuffle(x)
    np.random.shuffle(y)
    
    test_x, training_x = x[:n], x[n:]
    test_y, training_y = y[:n], y[n:]
    
    return training_x, training_y, test_x, test_y
    
    
def x_y(all_data, test_nodenames):
    x,y = [0], [0]
    while(len(x) < 10 or len(y) <10):
        name = np.random.choice(test_nodenames, 1)
        x,y = give_me_x_and_y_per_node(all_data, name[0])
    return x, y


In [231]:
path_finals = [Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_viral500m.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_human9g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_archea1.4g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria30g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria58g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria88g.tsv"),
               Path("/Users/manuez42/Desktop/fonda/A2-job-granularity/MG-HIBF/general_results_fonda/general_results_bacteria125g.tsv")]





all_data = build_big(path_finals)
test_nodenames = np.unique(all_data['Node Name'].values)

x_node,y_node = x_y(all_data, test_nodenames)
node_training_x, node_training_y, node_test_x, node_test_y = train_and_test(x_node, y_node)


x,y = give_me_all_x_y(all_data)
training_x, training_y, test_x, test_y = train_and_test(x,y)


In [232]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="relu", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(0.001), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 23ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 935180.634250001
Mean Squared Error on Test Data: 1023992.2852539885
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 97
y_pred_test:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [233]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="relu", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(0.003), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 17ms/step
Accuracy for model: -10480.868
Loss: 0.0
Mean Squared Error on Training Data: 2078669.9240639957
Mean Squared Error on Test Data: 2113335.4003554583
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[ 936.731   ]
 [1456.2352  ]
 [1526.0436  ]
 [ 561.7139  ]
 [  86.042885]
 [ 353.91223 ]
 [ 504.89316 ]
 [1828.6548  ]
 [1420.5193  ]
 [ 652.62714 ]]  len= 97
y_pred_test:  [[ 665.61475]
 [2493.62   ]
 [1444.8711 ]
 [1389.6737 ]
 [2078.0168 ]
 [ 454.5662 ]
 [ 542.2325 ]
 [ 977.31726]
 [ 454.5662 ]
 [ 657.4975 ]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [234]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="relu", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(0.005), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 935180.634250001
Mean Squared Error on Test Data: 1023992.2852539885
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 97
y_pred_test:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [235]:
# potential
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="relu", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(0.007), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: -10480.868
Loss: 0.0
Mean Squared Error on Training Data: 766570.7599845941
Mean Squared Error on Test Data: 948894.0858251647
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[375.75708]
 [584.14923]
 [612.1519 ]
 [225.324  ]
 [ 34.51495]
 [141.96715]
 [202.53111]
 [733.54034]
 [569.82227]
 [261.79263]]  len= 97
y_pred_test:  [[ 267.00244]
 [1000.2823 ]
 [ 579.59064]
 [ 557.449  ]
 [ 833.5686 ]
 [ 182.34312]
 [ 217.5093 ]
 [ 392.03772]
 [ 182.34312]
 [ 263.7463 ]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [236]:
# potential
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="relu", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(0.009), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: -10480.868
Loss: 0.0
Mean Squared Error on Training Data: 747675.1212549146
Mean Squared Error on Test Data: 901532.7412887416
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[183.87035 ]
 [285.8435  ]
 [299.54614 ]
 [110.25847 ]
 [ 16.889303]
 [ 69.46921 ]
 [ 99.105156]
 [358.9455  ]
 [278.83282 ]
 [128.10378 ]]  len= 97
y_pred_test:  [[130.6531 ]
 [489.47113]
 [283.61282]
 [272.77817]
 [407.8926 ]
 [ 89.22651]
 [106.43448]
 [191.83699]
 [ 89.22651]
 [129.05977]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [237]:
# new model

In [238]:
# potential
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 19ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 4085328.31912269
Mean Squared Error on Test Data: 3462543.0671136156
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[ -798.72784 ]
 [-1241.6964  ]
 [-1301.2203  ]
 [ -478.95984 ]
 [  -73.366684]
 [ -301.77237 ]
 [ -430.51013 ]
 [-1559.2496  ]
 [-1211.2423  ]
 [ -556.4794  ]]  len= 97
y_pred_test:  [[ -567.5536 ]
 [-2126.2495 ]
 [-1232.0065 ]
 [-1184.941  ]
 [-1771.8745 ]
 [ -387.59756]
 [ -462.3485 ]
 [ -833.3347 ]
 [ -387.59756]
 [ -560.6322 ]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [239]:
# potential
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.002), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 22ms/step
Accuracy for model: -10480.868
Loss: 0.0
Mean Squared Error on Training Data: 758378.0416483784
Mean Squared Error on Test Data: 940044.9465092927
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[362.77084 ]
 [563.96094 ]
 [590.99585 ]
 [217.53676 ]
 [ 33.322105]
 [137.06073 ]
 [195.5316  ]
 [708.1891  ]
 [550.1291  ]
 [252.74503 ]]  len= 97
y_pred_test:  [[257.77478]
 [965.71234]
 [559.5599 ]
 [538.1834 ]
 [804.76025]
 [176.0413 ]
 [209.99214]
 [378.48883]
 [176.0413 ]
 [254.63118]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [240]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.003), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 30ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 3927379.5312029156
Mean Squared Error on Test Data: 3337817.3901060647
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[ -773.4386  ]
 [-1202.382   ]
 [-1260.0212  ]
 [ -463.79504 ]
 [  -71.043755]
 [ -292.21768 ]
 [ -416.87936 ]
 [-1509.8809  ]
 [-1172.8921  ]
 [ -538.86017 ]]  len= 97
y_pred_test:  [[ -549.58374]
 [-2058.9282 ]
 [-1192.9989 ]
 [-1147.4236 ]
 [-1715.7737 ]
 [ -375.32547]
 [ -447.7097 ]
 [ -806.94977]
 [ -375.32547]
 [ -542.8815 ]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [241]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.006), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 19ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 1943162.9778525447
Mean Squared Error on Test Data: 1784319.903681159
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[-375.4933 ]
 [-583.73914]
 [-611.72217]
 [-225.16583]
 [ -34.49072]
 [-141.8675 ]
 [-202.38893]
 [-733.0254 ]
 [-569.42224]
 [-261.60886]]  len= 97
y_pred_test:  [[-266.815  ]
 [-999.5801 ]
 [-579.1838 ]
 [-557.0576 ]
 [-832.9834 ]
 [-182.21512]
 [-217.3566 ]
 [-391.7625 ]
 [-182.21512]
 [-263.56116]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [242]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.008), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 978008.5088850552
Mean Squared Error on Test Data: 1054618.4031123684
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[-26.043594 ]
 [-40.487183 ]
 [-42.42804  ]
 [-15.61713  ]
 [ -2.3922193]
 [ -9.839695 ]
 [-14.037362 ]
 [-50.84143  ]
 [-39.494186 ]
 [-18.144758 ]]  len= 97
y_pred_test:  [[-18.505848]
 [-69.32922 ]
 [-40.17123 ]
 [-38.636597]
 [-57.774353]
 [-12.63814 ]
 [-15.075495]
 [-27.172   ]
 [-12.63814 ]
 [-18.280167]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [243]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: -10480.868
Loss: 0.0
Mean Squared Error on Training Data: 855915.730451975
Mean Squared Error on Test Data: 968722.711591734
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[ 56.688206 ]
 [ 88.12707  ]
 [ 92.35167  ]
 [ 33.993275 ]
 [  5.2070622]
 [ 21.417727 ]
 [ 30.554647 ]
 [110.66481  ]
 [ 85.965645 ]
 [ 39.495075 ]]  len= 97
y_pred_test:  [[ 40.281048]
 [150.90656 ]
 [ 87.43935 ]
 [ 84.09897 ]
 [125.75546 ]
 [ 27.509007]
 [ 32.814316]
 [ 59.144367]
 [ 27.509007]
 [ 39.789814]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [244]:
# new model

In [245]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 19ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 3473770.3059813655
Mean Squared Error on Test Data: 2980235.900213219
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[ -697.1157 ]
 [-1083.731  ]
 [-1135.6824 ]
 [ -418.0278 ]
 [  -64.03316]
 [ -263.38168]
 [ -375.74173]
 [-1360.8859 ]
 [-1057.1512 ]
 [ -485.68546]]  len= 97
y_pred_test:  [[ -495.35086]
 [-1855.7534 ]
 [-1075.2738 ]
 [-1034.1959 ]
 [-1546.4612 ]
 [ -338.2884 ]
 [ -403.52972]
 [ -727.32   ]
 [ -338.2884 ]
 [ -489.31   ]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [246]:
# potential
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.003), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: -10480.868
Loss: 0.0
Mean Squared Error on Training Data: 778124.9805261891
Mean Squared Error on Test Data: 918201.5795093687
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[134.67647 ]
 [209.36707 ]
 [219.40361 ]
 [ 80.7592  ]
 [ 12.370629]
 [ 50.882965]
 [ 72.58992 ]
 [262.9109  ]
 [204.23209 ]
 [ 93.830055]]  len= 97
y_pred_test:  [[ 95.69732 ]
 [358.51483 ]
 [207.7332  ]
 [199.79733 ]
 [298.76236 ]
 [ 65.35426 ]
 [ 77.958305]
 [140.51167 ]
 [ 65.35426 ]
 [ 94.53028 ]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [247]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 19ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 1317435.4814445875
Mean Squared Error on Test Data: 1305833.145419222
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[-181.43037 ]
 [-282.05032 ]
 [-295.57114 ]
 [-108.795334]
 [ -16.665182]
 [ -68.54735 ]
 [ -97.79003 ]
 [-354.18228 ]
 [-275.13272 ]
 [-126.40383 ]]  len= 97
y_pred_test:  [[-128.91933 ]
 [-482.97583 ]
 [-279.84927 ]
 [-269.15842 ]
 [-402.47986 ]
 [ -88.042465]
 [-105.02209 ]
 [-189.2913  ]
 [ -88.042465]
 [-127.347145]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [248]:
# potential
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.007), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 30ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 2320739.950786313
Mean Squared Error on Test Data: 2077126.4843409613
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[-468.09436 ]
 [-727.6961  ]
 [-762.5801  ]
 [-280.69437 ]
 [ -42.996536]
 [-176.85368 ]
 [-252.30043 ]
 [-913.7981  ]
 [-709.84845 ]
 [-326.12466 ]]  len= 97
y_pred_test:  [[ -332.61472]
 [-1246.0883 ]
 [ -722.0173 ]
 [ -694.43463]
 [-1038.4069 ]
 [ -227.1515 ]
 [ -270.9593 ]
 [ -488.37576]
 [ -227.1515 ]
 [ -328.55844]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [249]:
# potential
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 17ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 2535101.6664771894
Mean Squared Error on Test Data: 2244174.2446237947
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[ -515.78375 ]
 [ -801.8336  ]
 [ -840.2716  ]
 [ -309.29144 ]
 [  -47.377014]
 [ -194.87149 ]
 [ -278.00476 ]
 [-1006.8957  ]
 [ -782.1677  ]
 [ -359.3502  ]]  len= 97
y_pred_test:  [[ -366.50143]
 [-1373.0396 ]
 [ -795.5763 ]
 [ -765.1835 ]
 [-1144.1996 ]
 [ -250.29366]
 [ -298.56458]
 [ -538.13135]
 [ -250.29366]
 [ -362.0319 ]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [250]:
# potential
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="relu", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: -10480.868
Loss: 0.0
Mean Squared Error on Training Data: 755634.9391201003
Mean Squared Error on Test Data: 937016.0031079067
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[358.02588 ]
 [556.5844  ]
 [583.2657  ]
 [214.69142 ]
 [ 32.886257]
 [135.268   ]
 [192.97408 ]
 [698.9261  ]
 [542.93353 ]
 [249.43916 ]]  len= 97
y_pred_test:  [[254.40314]
 [953.081  ]
 [552.24097]
 [531.1441 ]
 [794.23413]
 [173.73872]
 [207.24548]
 [373.53827]
 [173.73872]
 [251.30066]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [251]:
# potential
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="relu", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.003), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 935180.634250001
Mean Squared Error on Test Data: 1023992.2852539885
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 97
y_pred_test:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [252]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="relu", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 19ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 935180.634250001
Mean Squared Error on Test Data: 1023992.2852539885
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 97
y_pred_test:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [253]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="relu", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.007), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 935180.634250001
Mean Squared Error on Test Data: 1023992.2852539885
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 97
y_pred_test:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [254]:
# potential
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="linear", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.008), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 4388549.74323948
Mean Squared Error on Test Data: 3702253.402065436
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[ -845.6616 ]
 [-1314.6593 ]
 [-1377.6809 ]
 [ -507.10382]
 [  -77.67776]
 [ -319.50473]
 [ -455.8072 ]
 [-1650.8722 ]
 [-1282.4158 ]
 [ -589.17847]]  len= 97
y_pred_test:  [[ -600.9034 ]
 [-2251.1892 ]
 [-1304.4    ]
 [-1254.569  ]
 [-1875.9911 ]
 [ -410.37305]
 [ -489.51642]
 [ -882.30206]
 [ -410.37305]
 [ -593.5753 ]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [255]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1,)))
model.add(tf.keras.layers.Dense(1, activation="relu", use_bias=True))
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

accuracy, loss = model.evaluate(node_training_x, node_training_y, verbose=0)
y_pred_training = model.predict(node_training_x)
y_pred_test = model.predict(node_test_x)

mse_training = mean_squared_error(node_training_y, y_pred_training)
mse_test = mean_squared_error(node_test_y, y_pred_test)

print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Training Data:", mse_training)
print("Mean Squared Error on Test Data:", mse_test)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred_training: ",y_pred_training[:10], " len=", len(y_pred_training))
print("y_pred_test: ",y_pred_test[:10], " len=", len(y_pred_test))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

1/1 [==============================] - 0s 18ms/step
Accuracy for model: 10617.060
Loss: 0.0
Mean Squared Error on Training Data: 935180.634250001
Mean Squared Error on Test Data: 1023992.2852539885
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred_training:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 97
y_pred_test:  [[0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]
 [0.]]  len= 24
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [256]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=1, input_shape=(1,), activation='linear', use_bias=True)
])
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.1), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=100, verbose=0, validation_split=0.2)

loss = model.evaluate(x, y, verbose=0)
y_pred = model.predict(x)
mse = mean_squared_error(y, y_pred)

# print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Test Data:", mse)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred: ",y_pred[:10], " len=", len(y_pred))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

186/186 [==============================] - 0s 856us/step
Loss: [9394.791015625, 0.00016886186494957656]
Mean Squared Error on Test Data: 1146948.6240908646
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred:  [[  -9.687238 ]
 [  -5.312356 ]
 [-119.05927  ]
 [-288.7422   ]
 [-274.05508  ]
 [-291.8671   ]
 [  -7.1873055]
 [-607.9835   ]
 [-122.80917  ]
 [-415.9887   ]]  len= 5922
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97


In [257]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=4, activation='relu', input_shape=(1,)),
    tf.keras.layers.Dense(units=1, activation='relu')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(node_training_x, node_training_y, epochs=1000, verbose=0, validation_split=0.2)

loss = model.evaluate(x, y, verbose=0)
y_pred = model.predict(x)
mse = mean_squared_error(y, y_pred)

# print("Accuracy for model:", "%0.3f" % accuracy)
print("Loss:", loss)
print("Mean Squared Error on Test Data:", mse)
print("X: ", node_training_x[:10], " len=", len(node_training_x))
print("y_pred: ",y_pred[:10], " len=", len(y_pred))
print("Y: ",node_training_y[:10], " len=", len(node_training_y))

186/186 [==============================] - 0s 840us/step
Loss: [-9055.3271484375, 0.0]
Mean Squared Error on Test Data: 671866.3192935464
X:  [ 577.   897.   940.   346.    53.   218.   311.  1126.4  875.   402. ]  len= 97
y_pred:  [[  2.5163097]
 [  1.3915347]
 [ 30.63567  ]
 [ 74.26084  ]
 [ 70.48482  ]
 [ 75.06425  ]
 [  1.8735809]
 [156.33727  ]
 [ 31.599766 ]
 [106.97571  ]]  len= 5922
Y:  [ 109.68     14.3855 2318.7432  729.0232  775.3111 1388.9738 1550.2128
 1550.2128  142.3596  350.5074]  len= 97
